<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/static/images/logo-v4.svg" width=200/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/annotation_import/basics.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/tree/master/examples/annotation_import/basics.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Annotation Imports
* This notebook is a high level introduction demonstrating multiple ways to upload your annotations. This will cover the following:
    * Model-assisted labeling - used to provide pre-annotated data for your labelers. This will enable a reduction in the total amount of time to label your assets. Model-assisted labeling does not submit the labels automatically, and will need to be reviewed by a labeler for submission.
    * Label Import is used to provide ground truth labels. These can in turn be used and compared against prediction labels, or used as benchmarks to see how your labelers are doing.



* For information on what types of annotations are supported per data type, refer to this [documentation](https://docs.labelbox.com/docs/model-assisted-labeling#option-1-import-via-python-annotation-types-recommended)

* Notes:
    * If you are importing more than 1,000 mask annotations at a time, consider submitting separate jobs, as they can take longer than other annotation types to import.
    * Wait until the import job is complete before opening the Editor to make sure all annotations are imported properly.

# Installs

In [ ]:
!pip install -q 'labelbox[data]'

# Imports

In [ ]:
from typing import List
from labelbox import Client, LabelingFrontend, LabelImport, MALPredictionImport, OntologyBuilder, Tool, MediaType
from labelbox.schema.queue_mode import QueueMode
from labelbox.data.annotation_types import (
    Label, ImageData, ObjectAnnotation, Rectangle, Point, LabelList
)
from labelbox.data.serialization import NDJsonConverter
import uuid


3.28.0


# API Key and Client
Provide a valid api key below in order to properly connect to the Labelbox Client.

In [ ]:
# Add your api key
API_KEY= None
client = Client(api_key=API_KEY)

---- 
### Steps
1. Make sure project is setup
2. Collect annotations
3. Upload

### Project setup

We will be creating two projects, one for model-assisted labeling, and one for label imports

In [ ]:
# Project defaults to batch mode with benchmark quality settings queue_mode argument is not provided
mal_project = client.create_project(name="mal_annotation_import_demo",
                                    queue_mode=QueueMode.Batch, 
                                    auto_audit_percentage=1,
                                    auto_audit_number_of_labels=1,
                                    media_type=MediaType.Image)

li_project = client.create_project(name="label_import_project_demo",
                                    queue_mode=QueueMode.Batch,
                                    auto_audit_percentage=1,
                                    auto_audit_number_of_labels=1,
                                    media_type=MediaType.Image)

# Confirm succesfull creation of project 
print("Project Name: ", mal_project.name , " Project Id: ", mal_project.uid)
print( "Project Name:", li_project.name , "Project Id: ", li_project.uid) 


######################### DATASET CONSENSUS OPTION ########################
#Note that dataset base projects will be deprecated in the near future.

#To use Datasets/Consensus instead of Batches/Benchmarks use the following query: 
#In this case, 10% of all data rows need to be annotated by three labelers.

# dataset_project = client.create_project(name="datasets-test-project",
#                                 description="a description",
#                                 media_type=MediaType.Image,
#                                 auto_audit_percentage=0.1,
#                                 auto_audit_number_of_labels=3,
#                                 queue_mode=QueueMode.Dataset)

# dataset_project.datasets.connect(dataset)


Project Name:  mal_annotation_import_demo  Project Id:  cl9sjj4m85by408z508t0gzva
Project Name: label_import_project_demo Project Id:  cl9sjj4wtcwvh07xy3zisa57k


In [ ]:

dataset = client.create_dataset(name="annotation_import_demo_dataset")

test_imgs_url = [
    {
        "row_data": "https://storage.googleapis.com/labelbox-datasets/mapillary-traffic/images/--0le83jkA7Hq7N1fvIvTw.jpg",
        "global_key": "TEST-ID-%id" % uuid.uuid1() 
    },
    {
        "row_data": "https://storage.googleapis.com/labelbox-datasets/mapillary-traffic/images/--48MAqc82-bZdgGpaiexA.jpg",
        "global_key": "TEST-ID-%id" % uuid.uuid1() 
    },
    {
        "row_data": "https://storage.googleapis.com/labelbox-datasets/mapillary-traffic/images/--A4b2SOWVi4KL_ryAAtTg.jpg",
        "global_key": "TEST-ID-%id" % uuid.uuid1() 
    },
    {
        "row_data": "https://storage.googleapis.com/labelbox-datasets/mapillary-traffic/images/--DuAQ9qBB2mmsC7hV2Kzg.jpg",
        "global_key": "TEST-ID-%id" % uuid.uuid1()
    },
    {
        "row_data": "https://storage.googleapis.com/labelbox-datasets/mapillary-traffic/images/--FValmNpFJ8yo8X7uWODA.jpg",
        "global_key": "TEST-ID-%id" % uuid.uuid1()
    }

]


data_rows = dataset.create_data_rows(test_imgs_url)
data_rows.wait_till_done()
print(data_rows.failed_data_rows)
print("ERRORS: " , data_rows.errors)
print("RESULT URL: ", data_rows.result_url)

None
ERRORS:  None
RESULT URL:  https://storage.labelbox.com/cl3ahv73w1891087qbwzs3edd%2Fdata-row-imports-results%2Fcl9sjj6m62sq707yd7v3v1v5e_cl9sjj7iicww607xycahnhu7k.json?Expires=1667050751352&KeyName=labelbox-assets-key-3&Signature=VLFJXZ-8VdrnECOEzf-bJgxOQlc


In [ ]:
# Setup Batches and Ontology

# We need the data row ID to create a batch
batch_datarows = [dr.uid for dr in list(dataset.export_data_rows())]

# Create a batch to send to your MAL project
batch_mal = mal_project.create_batch(
  "first-batch-MAL-demo", # Each batch in a project must have a unique name
  batch_datarows, # A list of data rows or data row ids
  5 # priority between 1(Highest) - 5(lowest)
)

# Create a batch to send to your LI project
batch_li = li_project.create_batch(
    "first-batch-LI-demo", # Each batch in a project must have a unique name
    batch_datarows, # A list of data rows or data row ids
    5 # priority between 1(Highest) - 5(lowest)
)

# Setup your ontology / labeling editor
# Only update this if you have an on-prem deployment
ontology_builder = OntologyBuilder(tools=[
    Tool(tool=Tool.Type.BBOX, name="box")
])

editor = next(client.get_labeling_frontends(where=LabelingFrontend.name == "Editor")) # Unless using a custom editor,
# Connect your ontology and editor to your MAL and LI project
mal_project.setup(editor, ontology_builder.asdict())
li_project.setup(editor, ontology_builder.asdict())

print("Batch Li: ", batch_li)
print("Batch Mal: ", batch_mal)

Batch Li:  <Batch {'created_at': datetime.datetime(2022, 10, 28, 13, 39, 21, tzinfo=datetime.timezone.utc), 'name': 'first-batch-LI-demo', 'size': 5, 'uid': 'edff9710-56c5-11ed-9725-219767d59008', 'updated_at': datetime.datetime(2022, 10, 28, 13, 39, 21, tzinfo=datetime.timezone.utc)}>
Batch Mal:  <Batch {'created_at': datetime.datetime(2022, 10, 28, 13, 39, 20, tzinfo=datetime.timezone.utc), 'name': 'first-batch-MAL-demo', 'size': 5, 'uid': 'ecd4aba0-56c5-11ed-869b-eb1bbcf890fd', 'updated_at': datetime.datetime(2022, 10, 28, 13, 39, 20, tzinfo=datetime.timezone.utc)}>


#### Create Label using Annotation Type Objects
* It is recommended to use the Python SDK's annotation types. Below is an example of a bounding box, which is the Rectangle annotation type

* A more in depth example can be found [here](https://docs.labelbox.com/docs/bounding-box-json)

In [18]:
# Create a label with the related data and annotations
mal_onto =  mal_project.ontology().tools()
li_onto = li_project.ontology().tools()

## create an annotation with schema ids from both ontologies
rectangle = Rectangle(start=Point(x=100,y=100), end=Point(x=600,y=600))

rectangle_annotation = ObjectAnnotation(value=rectangle, name="box",feature_schema_id=mal_onto[0].feature_schema_id) 
rectangle_annotation_li = ObjectAnnotation(value=rectangle, name="box",feature_schema_id=li_onto[0].feature_schema_id) 

### Model Assisted Labeling 

To do model-assisted labeling, we need to convert a Label object into an NDJSON. 

This is easily done with using the NDJSONConverter class

We will create a Label called mal_label which has the same original structure as the label above

Notes:
* the NDJsonConverter takes in a list of labels

In [ ]:

# Create a list of labels

mal_ndjson_ls  = []

for data_row in list(mal_project.export_queued_data_rows()):
  image_data = ImageData(uid=data_row['id'])

  mal_label = Label(
      data=image_data,
      annotations = [rectangle_annotation]
  )
  mal_ndjson_ls.append(mal_label)

mal_ndjson = list(NDJsonConverter.serialize(mal_ndjson_ls))
## Create your ndjson list of of labels
mal_ndjson


[{'uuid': '5babc28e-f41b-405a-9e7e-cd1063e357f6',
  'dataRow': {'id': 'cl9sjj7qapr4p085n11igcvee'},
  'name': 'box',
  'schemaId': 'cl9sjji65cx0307xy29p24i3y',
  'classifications': [],
  'bbox': {'top': 100.0, 'left': 100.0, 'height': 500.0, 'width': 500.0}},
 {'uuid': 'e3e3f68e-4d5d-4d36-904e-165abfa15285',
  'dataRow': {'id': 'cl9sjj7q9pr4l085n7dzp5d93'},
  'name': 'box',
  'schemaId': 'cl9sjji65cx0307xy29p24i3y',
  'classifications': [],
  'bbox': {'top': 100.0, 'left': 100.0, 'height': 500.0, 'width': 500.0}},
 {'uuid': 'f36ec573-515f-4ed4-b09c-fddad3b8e695',
  'dataRow': {'id': 'cl9sjj7q9pr4h085n6fhz285p'},
  'name': 'box',
  'schemaId': 'cl9sjji65cx0307xy29p24i3y',
  'classifications': [],
  'bbox': {'top': 100.0, 'left': 100.0, 'height': 500.0, 'width': 500.0}},
 {'uuid': 'f5373bc3-29aa-492b-9305-c3a23b64d78b',
  'dataRow': {'id': 'cl9sjj7q9pr4d085nelq436ai'},
  'name': 'box',
  'schemaId': 'cl9sjji65cx0307xy29p24i3y',
  'classifications': [],
  'bbox': {'top': 100.0, 'left': 10

In [ ]:
upload_job = MALPredictionImport.create_from_objects(
    client = client, 
    project_id = mal_project.uid, 
    name="upload_mal_import_job_demo", 
    predictions=mal_ndjson)

In [ ]:
# Errors will appear for each annotation that failed.
# This will provide information only after the upload_job is complete, so we do not need to worry about having to rerun
upload_job.wait_until_done()
print("Errors:", upload_job.errors)

Errors: []


### Label Import

Label import is very similar to model-assisted labeling. We will create a Label called li_label which has the same original structure as the label above

In [19]:
li_label_list = []

for data_row in list(li_project.export_queued_data_rows()):
  image_data = ImageData(uid=data_row['id'])

  li_label = Label(
      data=image_data,
      annotations = [rectangle_annotation_li]
  )
  li_label_list.append(li_label)

## Create your ndjson list of of labels
li_ndjson = list(NDJsonConverter.serialize(li_label_list))

li_ndjson

[{'uuid': '2376c909-cbc2-4905-b084-e25b7d21f367',
  'dataRow': {'id': 'cl9sjj7qapr4p085n11igcvee'},
  'name': 'box',
  'schemaId': 'cl9sjjjvt7ent07wxd7br30oh',
  'classifications': [],
  'bbox': {'top': 100.0, 'left': 100.0, 'height': 500.0, 'width': 500.0}},
 {'uuid': 'd4b6ea84-0130-4302-9a2c-f3364991d7ac',
  'dataRow': {'id': 'cl9sjj7q9pr4l085n7dzp5d93'},
  'name': 'box',
  'schemaId': 'cl9sjjjvt7ent07wxd7br30oh',
  'classifications': [],
  'bbox': {'top': 100.0, 'left': 100.0, 'height': 500.0, 'width': 500.0}},
 {'uuid': 'f8cdc6c8-f7bc-4500-8385-d796dae7753c',
  'dataRow': {'id': 'cl9sjj7q9pr4h085n6fhz285p'},
  'name': 'box',
  'schemaId': 'cl9sjjjvt7ent07wxd7br30oh',
  'classifications': [],
  'bbox': {'top': 100.0, 'left': 100.0, 'height': 500.0, 'width': 500.0}},
 {'uuid': '1ccc9a86-fd65-4057-ae57-709a95a0c20f',
  'dataRow': {'id': 'cl9sjj7q9pr4d085nelq436ai'},
  'name': 'box',
  'schemaId': 'cl9sjjjvt7ent07wxd7br30oh',
  'classifications': [],
  'bbox': {'top': 100.0, 'left': 10

In [21]:
upload_job = LabelImport.create_from_objects(
    client = client, 
    project_id = li_project.uid, 
    name="upload_label_import_job_demo", 
    labels=li_ndjson)

In [22]:
upload_job.wait_until_done()
print("Errors:", upload_job.errors)


Errors: []
